In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
import scipy
from scipy.optimize import minimize

np.seterr(invalid='ignore')


#def negPoissontDist(n, lambda_val):
 #   return -np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n)

#def maximize(n):
 #   result = scipy.optimize.minimize(negPoissontDist, x0=[10], args=(n,))
  #return result


#This function gives the probability of getting n counts for a given bin
def poissant_distribution(n, lambda_val):
    return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore

#This is just so we can use the minimize function from scipy
def negative_probability_distribution(lambda_val, n):
    return -poissant_distribution(n, lambda_val)

#This will find the lambda that gives the highest probability of getting a given n, n would be our measured counts per bin
def maximize_distribution(n):
    result = scipy.optimize.minimize(negative_probability_distribution, x0=[10], args=(n,))
    return result.x

fixed_n = 13

optimal_lambda = maximize_distribution(fixed_n)
print(f"Lambda: {optimal_lambda}")



Lambda: [12.99900562]


In [2]:
import pandas as pds

#importing and sorting data
ge_data = pds.read_csv("SuperCDMS/PhysRevD.99.062001-data/MarchAprilFinal.txt", skiprows=1, \
                         names=['time', 'blah'], \
                         delim_whitespace=False
                     )

ge_data = ge_data.sort_values(by='time')

print (ge_data.head(10))

t = np.asarray(ge_data["time"], dtype=np.float32)

print (t)

        time    blah
9   506.1574  5.2894
5   507.4013  5.9058
8   509.4632  5.6750
10  509.8995  5.2232
13  510.0812  5.0349
3   512.5145  6.3243
19  512.7067  4.2205
12  513.7324  5.0628
14  513.8779  4.9122
4   514.5219  6.1499
[ 506.1574  507.4013  509.4632  509.8995  510.0812  512.5145  512.7067
  513.7324  513.8779  514.5219  516.8201  517.7107  518.5185  519.2819
  519.8351  524.4211  525.2859  525.2859  526.9375  528.7008  529.4175
  530.1161  534.9125  535.2761  536.9121  537.7742  543.2328  550.717
  551.226   553.132   553.3139  553.7683  554.0956  557.6922  558.5466
  565.1764  566.9474  567.3552  571.5335  574.8757  576.3014  578.0828
  578.1919  580.6901  583.0324  584.7126  587.0472  593.4147  593.986
  594.7132  596.1207  602.9245  616.2568  616.7476  623.1592  623.5592
  627.683   627.9011  637.3381  637.8289  638.3016  638.3275  648.4008
  651.4703  651.517   652.3974  652.5064  660.6087  661.3799  661.4708
  665.4129  670.0093  674.8498  676.0211  678.1193  680.363  

In [3]:
min_t=np.min(t)

#construct histogram from our data, every bin is roughly 12 hours
counts, bins = np.histogram(t-min_t,bins=72)
thing = (bins[:-1]+bins[1:])/2
error1 = [0.00,0.37,0.74,1.10,2.34,2.75,3.82,4.25,5.30,6.33,6.78,7.81,8.83,9.28]
error2 = [1.29,2.75,4.25,5.30,6.78,7.81,9.28,10.30,11.32,12.79,13.81,14.82,16.29,17.30]
ntot = counts
ntot_plus = np.zeros(np.shape(ntot))
ntot_minus = np.zeros(np.shape(ntot))
for i,ncount in enumerate(ntot):
    if ncount<=20:
        ntot_plus[i] = error2[ncount]-ncount
        ntot_minus[i] = ncount-error1[ncount]
    else:
        ntot_plus[i] = np.sqrt(ncount)
        ntot_minus[i] = np.sqrt(ncount)

#probabilities = np.array([poissant_distribution(n, 2) for n in counts]) i don't know what i was doing here

#optimal_lambda = maximize_distribution(counts, probabilities)


In [4]:
print(counts)


[13  9  5  6  5  7  6  1  2  4  4  5  5  5  6  2  3  3  0  3  3  5  5  4
  3  4  2  3  1  3  4  0  3  1  3  4  2  3  1  2  2  1  2  2  1  2  1  1
  3  2  1  0  0  1  1  1  1  1  1  2  1  2  0  1  0  2  1  1  1  1  1  2]


In [5]:
#initializing the array of optimal lambdas
lambda_array = []
for i in range(len(counts)):
    result=maximize_distribution(counts[i])
    print(result)
    lambda_array.extend(result)
# 
prob_array = []
for i in range(len(lambda_array)):
    result=poissant_distribution(counts[i], lambda_array[i])
    print(result)
    prob_array.extend([result])

print(prob_array)

[12.99900562]
[9.00022898]
[5.00012877]
[6.83768424]
[5.00012877]
[7.00004227]
[6.83768424]
[0.99999105]
[2.00004861]
[4.18880904]


C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in power
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\15591339

[4.18880904]
[5.00012877]
[5.00012877]
[5.00012877]
[6.83768424]
[2.00004861]
[2.99990993]
[2.99990993]
[9.95351047]
[2.99990993]
[2.99990993]
[5.00012877]
[5.00012877]
[4.18880904]
[2.99990993]
[4.18880904]
[2.00004861]
[2.99990993]
[0.99999105]
[2.99990993]

C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in power
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\15591339


[4.18880904]
[9.95351047]
[2.99990993]
[0.99999105]
[2.99990993]
[4.18880904]
[2.00004861]
[2.99990993]
[0.99999105]
[2.00004861]
[2.00004861]
[0.99999105]
[2.00004861]
[2.00004861]
[0.99999105]
[2.00004861]
[0.99999105]
[0.99999105]
[2.99990993]
[2.00004861]
[0.99999105]

C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941


[9.95351047]
[9.95351047]
[0.99999105]
[0.99999105]
[0.99999105]
[0.99999105]
[0.99999105]
[0.99999105]
[2.00004861]
[0.99999105]
[2.00004861]
[9.95351047]
[0.99999105]
[9.95351047]
[2.00004861]
[0.99999105]
[0.99999105]
[0.99999105]
[0.99999105]
[0.99999105]
[2.00004861]
0.10993981006713183
0.1317556396257346
0.1754673694769224
0.1522485685707315
0.1754673694769224
0.14900277965532002
0.1522485685707315
0.36787944115671767
0.27067056631334846
0.19452452122114908
0.19452452122114908
0.1754673694769224
0.1754673694769224
0.1754673694769224
0.1522485685707315
0.27067056631334846
0.22404180735245657
0.22404180735245657
4.756038115004805e-05
0.22404180735245657
0.22404180735245657
0.1754673694769224
0.1754673694769224
0.19452452122114908
0.22404180735245657
0.19452452122114908
0.27067056631334846
0.22404180735245657
0.36787944115671767
0.22404180735245657
0.19452452122114908
4.756038115004805e-05
0.22404180735245657
0.36787944115671767
0.22404180735245657
0.19452452122114908
0.27067056631

C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore
C:\Users\cecih\AppData\Local\Temp\ipykernel_41004\1559133941.py:21: RuntimeWarning: overflow encountered in exp
  return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n) #numpy gets mad at the math here but can just ignore


In [6]:
#nevermind all this stuff but will be useful when we actually do the model function
# some function parameters can be initialized as constants?
#x = np.linspace(0,926,num=100) #i cant remember why we chose 926, workedwell for the other plots
#print (function(x,926,271.2,1))
#alpha = 274.32 #half life of germ 71 in hours
#tstop = 1440 #hours

#print(len(counts))
#print(len(prob_array))

counts1=np.asarray(counts)
prob1=np.asarray(prob_array)
# Model function is the poissant distribution
def likelihood_func(theta, n, prob): 
    lambda_val = theta
    model = (np.exp(-1*lambda_val)*(lambda_val)**n)/np.math.factorial(n)
    return model

lambda_true = 5.0
np.random.seed(69)
nll = lambda *args: -likelihood_func(*args)
initial = np.asarray([5]) 
soln = scipy.optimize.minimize(nll, initial, args=(counts1[1], prob1[1])) #I think it has a problem with the size of the arrays in the input of the likelihood function??
lambda_val_ml = soln.x
print(soln.x)
#idk


[8.99999635]


In [12]:
def prior(theta): #this sets the prior conditions of our parameters
    lambda_val = theta
    if lambda_val>0:
        return 0.0
    return -np.inf

def prob(theta, n):
    lp = prior(theta)
    if not np.isfinite(lp):
        return -np.inf 
    return lp + likelihood_func(theta, n, prob)

pos = soln.x + 1e-4 * np.random.randn(32, 3)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, prob, args=(counts1[0])
)

sampler.run_mcmc(pos, 5000, progress=True);

fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["lambda"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

#autocorrelation stuffs
tau = sampler.get_autocorr_time()
print(tau)

flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
print(flat_samples.shape)

fig = corner.corner(
    flat_samples, labels=labels, truths=[m_true, b_true, np.log(f_true)]
);

emcee: Exception while calling your likelihood function:
  params: [9.00017024 8.99986949 9.00009791]
  args: 13
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "C:\Users\cecih\.conda\envs\neutroncapture\lib\site-packages\emcee\ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
TypeError: Value after * must be an iterable, not numpy.intc


TypeError: Value after * must be an iterable, not numpy.intc